In [34]:
import pandas as pd
import os
import json
import sqlite3, sqlalchemy 

In [36]:
with open('../parameters.json') as json_data:
    params = json.load(json_data)
    
DATA_DIR = params['DIRECTORIES']['DATA_DIR']
TMP_DIR = params['DIRECTORIES']['TMP_DIR']
PRICE_FILE = params['FILES']['PRICE_FILE']    
STATION_FILE = params['FILES']['STATION_FILE']    
SERVICES_FILE = params['FILES']['SERVICES_FILE'] 
GAS_DB = os.path.join(DATA_DIR, params['DATABASES']['GAS_DB'])

In [3]:
gas_files = ['Prix2014.zip', 'Prix2015.zip', 'Prix2016.zip']
stations_file = 'Stations2016.zip'
services_file = 'Services2016.zip'

In [5]:
df2014 = pd.read_csv(os.path.join(DATA_DIR, gas_files[0]), 
                     sep=';', header=None, dtype= {1: str}) 
df2014.columns = ['id_station', 'code_postal', 'type_station',
                  'latitude', 'longitude', 'date_releve',
                  'type_carburant', 'libelle_carburant', 'prix']

In [6]:
df2014.head()

id_station code_postal type_station   latitude  longitude  \
0     1000001       01000            R  4620114.0   519791.0   
1     1000001       01000            R  4620114.0   519791.0   
2     1000001       01000            R  4620114.0   519791.0   
3     1000001       01000            R  4620114.0   519791.0   
4     1000001       01000            R  4620114.0   519791.0   

           date_releve  type_carburant libelle_carburant    prix  
0  2014-01-02 11:08:03             1.0            Gazole  1304.0  
1  2014-01-04 09:54:03             1.0            Gazole  1304.0  
2  2014-01-05 10:27:09             1.0            Gazole  1304.0  
3  2014-01-06 09:07:51             1.0            Gazole  1304.0  
4  2014-01-07 09:23:56             1.0            Gazole  1304.0

In [7]:
df2014.dtypes

id_station             int64
code_postal           object
type_station          object
latitude             float64
longitude            float64
date_releve           object
type_carburant       float64
libelle_carburant     object
prix                 float64
dtype: object

In [ ]:
dfall = pd.DataFrame()
for year in gas_files: 
    df = pd.read_csv(os.path.join(DATA_DIR, year), 
                     sep=';', header=None, dtype= {1: str}
                    )

    df.columns = ['id_station', 'code_postal', 'type_station',
                  'latitude', 'longitude', 'date_releve',
                  'type_carburant', 'libelle_carburant', 'prix']
    
    dfall = pd.concat([dfall, df], axis=0)

In [9]:
import glob

gas_files = glob.glob(os.path.join(DATA_DIR, "Prix*.zip"))

In [10]:
gas_files

['../../data\\Prix2014.zip',
 '../../data\\Prix2015.zip',
 '../../data\\Prix2016.zip']

In [11]:
%%timeit
dfall = pd.DataFrame()
for year in gas_files: 
    df = pd.read_csv(year, 
                     sep=';', header=None, dtype= {1: str}
                    )

    df.columns = ['id_station', 'code_postal', 'type_station',
                  'latitude', 'longitude', 'date_releve',
                  'type_carburant', 'libelle_carburant', 'prix']
    
    dfall = pd.concat([dfall, df], axis=0)

dfall.shape
assert dfall.shape[1] == 9

1 loop, best of 3: 11.6 s per loop


In [29]:
dfstations = pd.read_csv(os.path.join(DATA_DIR, stations_file),
                        sep='|',
                        header=None,
                        encoding='utf-8')
dfstations.columns = ['id_station', 'code_postal', 'type_station',
                  'latitude', 'longitude', 'adresse', 'ville']

In [13]:
dfstations.head()

id_station  code_postal type_station      latitude      longitude  \
0     1000001         1000            R  4.620114e+06  519791.000000   
1     1000002         1000            R  4.621842e+06  522767.000000   
2     1000004         1000            R  4.618836e+06  524447.000000   
3     1000005         1000            R  4.620094e+06  519942.024022   
4     1000006         1000            R  4.620754e+06  523758.000000   

                      adresse                  ville  
0       596 AVENUE DE TREVOUX  SAINT-DENIS-LèS-BOURG  
1         16 Avenue de Marboz        BOURG-EN-BRESSE  
2  20 Avenue du Maréchal Juin        Bourg-en-Bresse  
3       642 Avenue de Trévoux  SAINT-DENIS-LèS-BOURG  
4    1 Boulevard John Kennedy        BOURG-EN-BRESSE

In [30]:
dfservices = pd.read_csv(os.path.join(DATA_DIR, services_file),
                        sep='|',
                        dtype = {1:str},
                        header=None,
                        encoding='utf-8')
dfservices.columns = ['id_station', 'code_postal', 'type_station',
                  'latitude', 'longitude', 'services']  

In [18]:
dfservices.head()

id_station code_postal type_station      latitude      longitude  \
0     1000001       01000            R  4.620114e+06  519791.000000   
1     1000002       01000            R  4.621842e+06  522767.000000   
2     1000004       01000            R  4.618836e+06  524447.000000   
3     1000005       01000            R  4.620094e+06  519942.024022   
4     1000006       01000            R  4.620754e+06  523758.000000   

                                            services  
0  Station de gonflage,Vente de gaz domestique,Au...  
1                Vente de gaz domestique,Automate CB  
2  Carburant qualité supérieure,GPL,Toilettes pub...  
3  Carburant qualité supérieure,Station de lavage...  
4                            Vente de gaz domestique

In [32]:
dfall.to_pickle(os.path.join(DATA_DIR, PRICE_FILE))
dfstations.to_pickle(os.path.join(DATA_DIR, STATION_FILE))
dfservices.to_pickle(os.path.join(DATA_DIR, SERVICES_FILE)) 

In [38]:
cnx = sqlite3.connect(GAS_DB)

In [44]:
dfall.to_sql('prices', cnx, index= False, if_exists='replace')

In [45]:
dfstations.to_sql('stations', cnx, index= False, if_exists='replace')

In [46]:
dfservices.to_sql('services', cnx, index= False, if_exists='replace')

In [43]:
pd.read_sql_query('select * from prices where code_postal="13001"',
                 cnx)

id_station code_postal type_station   latitude  longitude  \
0      13001001       13001            R  4330446.0   538957.0   
1      13001001       13001            R  4330446.0   538957.0   
2      13001001       13001            R  4330446.0   538957.0   
3      13001001       13001            R  4330446.0   538957.0   
4      13001001       13001            R  4330446.0   538957.0   
5      13001001       13001            R  4330446.0   538957.0   
6      13001001       13001            R  4330446.0   538957.0   
7      13001001       13001            R  4330446.0   538957.0   
8      13001001       13001            R  4330446.0   538957.0   
9      13001001       13001            R  4330446.0   538957.0   
10     13001001       13001            R  4330446.0   538957.0   
11     13001001       13001            R  4330446.0   538957.0   
12     13001001       13001            R  4330446.0   538957.0   
13     13001001       13001            R  4330446.0   538957.0   
14     13001001       13001            R  4330446.0   538957.0   
15     13001001       13001            R  4330446.0   538957.0   
16     13001001       13001            R  4330446.0   538957.0   
17     13001001       13001            R  4330446.0   538957.0   
18     13001001       13001            R  4330446.0   538957.0   
19     13001001       13001            R  4330446.0   538957.0   
20     13001001       13001            R  4330446.0   538957.0   
21     13001001       13001            R  4330446.0   538957.0   
22     13001001       13001            R  4330446.0   538957.0   
23     13001001       13001            R  4330446.0   538957.0   
24     13001001       13001            R  4330446.0   538957.0   
25     13001001       13001            R  4330446.0   538957.0   
26     13001001       13001            R  4330446.0   538957.0   
27     13001001       13001            R  4330446.0   538957.0   
28     13001001       13001            R  4330446.0   538957.0   
29     13001001       13001            R  4330446.0   538957.0   
..          ...         ...          ...        ...        ...   
153    13001001       13001            R  4330446.0   538957.0   
154    13001001       13001            R  4330446.0   538957.0   
155    13001001       13001            R  4330446.0   538957.0   
156    13001001       13001            R  4330446.0   538957.0   
157    13001001       13001            R  4330446.0   538957.0   
158    13001001       13001            R  4330446.0   538957.0   
159    13001001       13001            R  4330446.0   538957.0   
160    13001001       13001            R  4330446.0   538957.0   
161    13001001       13001            R  4330446.0   538957.0   
162    13001001       13001            R  4330446.0   538957.0   
163    13001001       13001            R  4330446.0   538957.0   
164    13001001       13001            R  4330446.0   538957.0   
165    13001001       13001            R  4330446.0   538957.0   
166    13001001       13001            R  4330446.0   538957.0   
167    13001001       13001            R  4330446.0   538957.0   
168    13001001       13001            R  4330446.0   538957.0   
169    13001001       13001            R  4330446.0   538957.0   
170    13001001       13001            R  4330446.0   538957.0   
171    13001001       13001            R  4330446.0   538957.0   
172    13001001       13001            R  4330446.0   538957.0   
173    13001001       13001            R  4330446.0   538957.0   
174    13001001       13001            R  4330446.0   538957.0   
175    13001001       13001            R  4330446.0   538957.0   
176    13001001       13001            R  4330446.0   538957.0   
177    13001001       13001            R  4330446.0   538957.0   
178    13001001       13001            R  4330446.0   538957.0   
179    13001001       13001            R  4330446.0   538957.0   
180    13001001       13001            R  4330446.0   538957.0   
181    13001001       13001            R  433